In [1]:
import requests
import pandas as pd
import hashlib
import time
from pprint import pprint

In [15]:
def get_marvel_dataframe(public_key, hash, number_of_characters, start_string = ""):
    if(number_of_characters > 100):
        raise Exception('Please specify number of characters less than or equal to 100.')
        
    t = time.strftime('%Y%d%m%H%M%S')
    
    if(start_string == ""):
        response = requests.get('https://gateway.marvel.com:443/v1/public/characters?apikey={}&ts={}&hash={}&limit={}'.format(public_key, t, hash, number_of_characters))
    else:
        response = requests.get('https://gateway.marvel.com:443/v1/public/characters?apikey={}&ts={}&nameStartsWith={}&hash={}&limit={}'.format(public_key, t, start_string, hash, number_of_characters))
    
    raw_data = response.json()
    
#     print(raw_data)
    
    character_info = []
    for character in raw_data['data']['results']:
        character_info.append([character['name'], character['comics']['available'], character['events']['available'], 
                               character['series']['available'], character['stories']['available'], character['id']])
        
    marvels_database = pd.DataFrame(character_info, columns = ['name', 'number_of_comics', 'number_of_events',
                                                               'number_of_series', 'number_of_stories', 'character_id'])
    
    return marvels_database

In [19]:
private_key = '69b650321683071b76e2efd65d5c6f91ebf26200'
public_key = '2fa0da01bf13cd7322443828e55ba972'

m = hashlib.md5()
m.update("{}{}{}".format(time.strftime("%Y%d%m%H%M%S"), private_key, public_key).encode('utf-8'))
hash = m.hexdigest()

avengers = get_marvel_dataframe(public_key, hash, 50, 'Iron')

In [ ]:
def filter_string_data(dataframe, column, check):
    return dataframe[dataframe['column'] == check]

def filter_numeric_data(dataframe, column, more_than, less_than):
    return dataframe[dataframe['column'] > more_than and dataframe['column'] < less_than]